# Testing GPU issue

In [1]:
import numpy as np
from scipy.stats import multivariate_normal

from tqdm.notebook import trange

import parallelkdepy as pkde

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:
mu = np.zeros(2)
cov = np.identity(2)

rv = multivariate_normal(mean=mu, cov=cov, allow_singular=False)

In [3]:
def estimate_density(samples, grid):    
    density_estimation = pkde.DensityEstimation(samples, grid=grid, device="cuda")
    density_estimation.estimate_density("parallelEstimator", method="cuda")
    
    density_estimated = density_estimation.get_density()

    return density_estimated

### Grid and compile

In [4]:
grid_min = -8
grid_max = 8
grid_size = 500

grid = pkde.Grid([(grid_min, grid_max, grid_size)]*2, device="cuda")

In [5]:
samples = rv.rvs(size=10000)
density_estimated = estimate_density(samples, grid)

## Run loop

Remove pieces that call to julia until we find the one that crashes. Then work from there.

In [ ]:
n_reps = 1000
for _ in trange(n_reps):
    samples = rv.rvs(size=10000)
    density_estimated = estimate_density(samples, grid)

  0%|          | 0/1000 [00:00<?, ?it/s]

## Testing JuliaCall

In [1]:
import numpy as np
from juliacall import Main as jl

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [2]:
jl.seval("using ParallelKDE")
jl.seval("using CUDA")

In [3]:
arr_d = jl.CUDA.rand(10,10)

In [4]:
type(arr_d)

juliacall.ArrayValue

In [5]:
jl.typeof(arr_d)

CuArray{Float32, 2, CUDA.DeviceMemory}

In [6]:
jl.isa(arr_d, jl.CuArray)

True

In [8]:
jl.size(arr_d)

(10, 10)

In [9]:
arr_d.shape

(10, 10)

In [10]:
arr = jl.Array(arr_d)

In [11]:
arr_np = arr.to_numpy()